# Beer Score Data Exploration and Analysis
This notebook transforms data sourced from [https://data.world/socialmediadata/beeradvocate](https://data.world/socialmediadata/beeradvocate) as part of creating a beer prediction machine learning model.  The dataset is ~1.5 million reviews, spanning more than 10 years **(insert date range)**.  Each review includes ratings from [ratebeer](https://www.ratebeer.com/Story.asp?StoryID=103) across five "features": appearance, aroma, palate, taste, and overall impression.  A quick explanation of each rating follows however for a full explanation refer to the website.


### Index
* [Data Extraction](#dataextract)

* [Data Transformation](#datatransform)
    * [Exploratory Analysis](#dataexplore)
    * [Visualising the Data](#dataviz)
        
* [Adding Features](#dataadd)









### Beer Ratings
**Appearance:** (rating out of 5)
  After pouring rating includes observations on the visual appeal, including the colour, clarity, carbonation, and head       size and longevity, as well as the extent and pattern of lacing on the glass

**Aroma:** (rating out of 10)
  Any attractive, unusual or bad aromas. Hop character, malts, sweetness, fruitiness and other aromas including more subtle   aromas released after swirling the glass

**Palate:** (rating out of 5)
  The “feel” of the beer inside the mouth, at the front, the back and as you swallow, concentrating on the body or fullness   of the beer and any other special feature of how it feels in the mouth

**Flavour:** (rating out of 10) 
  How the beer tastes, the number of different tastes and flavours that can be identified.  Consideration also given to       variation in flavour from the start, to the middle, the finish and then the aftertaste of the beer. Assessment includes     intensity of the bitterness, sweetness and sourness of the beer

**Overall:** (rating out of 20) 
  A way of balancing up other features about the beer or anything else liked or disliked about it eg include price,           likelihood of buying again, etc





## Data Extraction <a id="dataextract"></a>


In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import time
sns.set()
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [2]:
# create dataframe
#beer_df = pd.read_csv("https://query.data.world/s/nuub3qupegsd33g3nimifjpajqeq2o")

# for local drive use
beer_df = pd.read_csv("data/beer.gz")



In [ ]:
# drop all rows with any NaN values
beer_df = beer_df.dropna()


In [ ]:
# remove duplicate rows in place
beer_df.drop_duplicates(keep="first")
beer_df = beer_df.reset_index(drop = True)

# show first 10 rows
beer_df.head(10)


In [ ]:
# save to csv file for local use
beer_data = beer_df.to_csv("data/beer.gz", header = True, compression="gzip", index = False)


## Data Transformation <a id="datatransform"></a>

### Exploratory analysis  <a id="dataexplore"></a>

An initial inspection of dataset is done for information on number of rows and columns, the data types and for confirmation that no null values remain.  A basic statistical analysis is then be completed using the **dataframe.describe()** to show for each feature, the:
- mean or average value
- standard deviation, which shows the spread of the data
- range of the data (min and max)
- 25%, 50% and 75% quartiles, showing skewedness in the data and the existence of outliers

Note as the default **DataFrame.describe()** method does not include categorical values in the summary, an additional check is done using the parameter _(include=[object])_ to examine the string columns.  This was chosen over the _(include="all")_ parameter to increase the readability of both summary outputs.

In [ ]:
# inspect number of rows and columns 
beer_df.shape


In [ ]:
# check additional information about dataframe ie spread of data types, null values, total number of records
beer_df.info()


In [ ]:
# convert epoch unix time as integer to timestamp for plotting
beer_df["review_time"] = pd.to_datetime(beer_df["review_time"], unit = "s")


In [ ]:
# check data type
beer_df["review_time"].dtypes


In [ ]:
# check endianess
np.dtype("datetime64[ns]") == np.dtype("<M8[ns]")


In [ ]:
# inspect basic statistic summary details (all numberic fields)
beer_df.describe().T


Preliminary observations made, based on the above include :
- **review_time** conversion to timestamp excludes the column from **DataFrame.describe()** method calculations.  This feature will need to be investigated independently
- **brewery_id** and **beer_beerid** are ordinal in nature as such no statistical inference can be made from the related statisics 
- the min values of zero for **review_overall** and **review_appearance** indicate beers in the dataset with no ratings.  These will need to be removed from the dataset
- with the exception of **review_overall** and **review_appearance** the remaining review ratings (review_aroma, review_appearance and review_taste) have ratings that fall within the range of 1-5 as expected and will not require additional cleaning
- the 25%, 50%, and 75% percentiles for all review ratings are largely consistent.  Worthnoting however is the complete absence of a 3rd quartile (75%) for **review_aroma**, **review_appearance** and **review_palate**.  In addition, with each feature having a mean that is lower than the median, there is a distinct skewness of these features to the right.  Whilst not as extreme, this is also the case for the remaining review features, **review_overall** and **review_taste**.  This will be confirmed later with a number of visual checks
- the max value for **beer_abv** is 57.7 which seems extremely high for an alcohol content and requires further investigation
- conversion of the **review_time** from an integer to a datetime format 


In [ ]:
# filter for review_overall greater than zero and confirm results
beer_df = beer_df[beer_df["review_overall"] > 0]


Filtering to exclude rows where the **review_overall** value is less than zero has also dealt with zero values for **review_appearance**.

In [ ]:
# check data type
beer_df["review_time"].dtypes


In [ ]:
# confirming endianess in environment
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')
    

### Visualising the data  <a id="dataviz"></a>

In [ ]:
# inspect distribution of dataframe numeric columns
beer_df.hist(figsize = (20,20));


In [ ]:
# inspect abv value skewness using a scatterplot
abv_check = beer_df.groupby(["beer_abv", "beer_name"]).size().reset_index(name="counts")

plt.scatter(abv_check["beer_abv"], abv_check["counts"])
plt.annotate("long tail of high abv beers",(25,1000));
plt.show()


In [ ]:
# slice dataframe to list beers with values >=20% abv
new_abv_check = abv_check.loc[abv_check["beer_abv"] >= 20].sort_values(
    "beer_abv", ascending = False)
new_abv_check


A validation of the abv for each individual beer (20 in total) confirmed that **beer_abv** values are correct and should remain in the dataset.

In [ ]:
# visualise check of review scores to identify outliers
beer_df.boxplot(column=["review_overall","review_aroma", "review_appearance","review_palate", "review_taste"],
                figsize = (15,6), return_type="axes", notch = True, 
                flierprops=dict(marker='s', markersize = 7, markerfacecolor="b"));

# add title and ticks
plt.title("Boxplot of Ratings", fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12);


Interestingly, there are 2 distinct groups within the 5 ratings features, which was not immediately evident from the initial histograms.  Also of note is that the proximity of the mean to the median in the group of **review_aroma**, **review_appearance** and **review_palate** is so small that there is no _'notching'_ at the median points in the plot.  This shows how heavily skewed the data is to the upper limit of the ratings, with the absence of the 75% quartile.

In [ ]:
# resize dataframe to analyse ratings
columns = ["beer_style","review_overall","review_aroma","review_appearance","review_palate","review_taste"]
reviews_df = beer_df.loc[:,columns]
#reviews_df = reviews_df.set_index("beer_style")

# plot heatmap to show correlations between the ratings features
plt.figure(figsize=(6,5))
sns.heatmap(reviews_df.corr(), xticklabels=reviews_df.corr().columns, 
            yticklabels=reviews_df.corr().columns, cmap="GnBu", center=0, annot=True)

# add title and ticks
plt.title("Heatmap of Ratings", fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


In [ ]:
review_counts = reviews_df.groupby(["beer_style"]).mean()
review_counts

In [ ]:
# Plot
plt.figure(figsize=(10,8));
sns.pairplot(review_counts, kind="scatter");
plt.show();


In [ ]:
# resize dataframe to analyse ratings
columns = ["review_profilename","review_overall","review_aroma","review_appearance","review_palate","review_taste"]
reviewer_counts = beer_df.loc[:,columns]

reviewer_counts = reviewer_counts.groupby(["review_profilename"]).mean()
reviewer_counts


In [ ]:
# Plot
plt.figure(figsize=(10,8));
sns.pairplot(reviewer_counts, kind="scatter");
plt.show();


In [ ]:
# check for large number of same reviewers ie reviewer bias
total_reviews = beer_df["review_profilename"].count()
reviewers = beer_df.groupby("review_profilename").size().reset_index(name="counts")
reviewers = reviewers.sort_values(by = "counts", ascending = False)

weighting = round(reviewers["counts"]/total_reviews * 100,2)
weighted_reviewers = pd.concat([reviewers, weighting], axis = 1)
weighted_reviewers.columns = ["review_profilename","ratings","% total"]
weighted_reviewers.head(15)


Now let's look at the categorical values

In [ ]:
beer_df.describe(include=[object])

In [ ]:
# save to csv file for import
beer_data = beer_df.to_csv("data/beer.gz", header = True, compression="gzip", index = False)


In [3]:
# extract unique breweries and beer names
brewery_beer_df = beer_df.loc[:,["brewery_id", "beer_beerid"]]
brewery_beer_df = brewery_beer_df.drop_duplicates(subset=["brewery_id", "beer_beerid"], keep=False)

brewery_beer = []

for row in brewery_beer_df.itertuples():
    brewery_beer.append(str(row.brewery_id) + "/" + str(row.beer_beerid))
    


In [4]:
# create webdriver object
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
    

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\Michelle H\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [5]:
# scrape beer profile from beer advocate website for status

status_xpath = "/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[4]/div[1]/div[1]/div/div[2]/dl/dd[9]/span"
beer_xpath = "/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/input[2]"

beer_status = []
count = 0

for i in range(len(0, len(brewery_beer)):
    try:
        url = "https://www.beeradvocate.com/beer/profile/" + brewery_beer[i]
        browser.visit(url)
        status = browser.find_by_xpath(status_xpath).text
        beer = brewery_beer[i]
        beer_id = beer[beer.find("/")+1:]
        beer_status["Status"].append(status)
        beer_status["Beer_Beerid"].append(beer_id)
        count +=1
        print(beer_id)
        print(count)
    except:
        next        

    

22579
1
22563
2
22564
3
36777
4
40307
5
37343
6
76171
7
15504
8
15503
9
3705
10
40183
11
76096
12
27017
13
75377
14
47538
15
28263
16
12857
17
56505
18
31889
19
52458
20
24590
21
35932
22
76704
23
76691
24
59078
25
70203
26
53891
27
56049
28
18693
29
70225
30
27790
31
72931
32
61906
33
45553
34
43806
35
76845
36
46635
37
37478
38
68286
39
27267
40
28135
41
50238
42
44187
43
34508
44
56661
45
28870
46
59566
47
61513
48
27490
49
63134
50
30848
51
32155
52
33807
53
36658
54
63135
55
28136
56
15824
57
61315
58
56219
59
56048
60
49188
61
60114
62
38203
63
31570
64
26115
65
20287
66
31553
67
63095
68
63099
69
63093
70
63098
71
74040
72
63097
73
63096
74
7365
75
16392
76
2291
77
23372
78
21244
79
16394
80
43058
81
75641
82
75640
83
67854
84
75637
85
14510
86
75617
87
66626
88
60159
89
56673
90
60160
91
65242
92
76260
93
76074
94
54781
95
76889
96
46901
97
69818
98
71773
99
75242
100
62263
101
60504
102
32108
103
56671
104
25043
105
32104
106
45107
107
45808
108
45630
109
47215
110
45415
111
4

64261
834
61717
835
48787
836
74859
837
67757
838
34899
839
35037
840
70210
841
48217
842
74607
843
57967
844
38879
845
74824
846
31904
847
58783
848
22396
849
48877
850
75648
851
65118
852
70911
853
66361
854
77004
855
74691
856
72612
857
71380
858
28874
859
52034
860
55052
861
56031
862
73786
863
73619
864
68308
865
54956
866
77005
867
54957
868
63781
869
70755
870
67101
871
56145
872
54955
873
63780
874
67102
875
58097
876
69382
877
65722
878
66263
879
58096
880
76725
881
50207
882
49931
883
69967
884
34217
885
39298
886
39319
887
34216
888
39290
889
39309
890
39337
891
76225
892
77068
893
36358
894
36548
895
18504
896
36356
897
61145
898
70719
899
22389
900
40534
901
62361
902
36705
903
16068
904
16055
905
16150
906
26196
907
65914
908
67393
909
47531
910
44976
911
41641
912
49476
913
37802
914
49698
915
55873
916
55266
917
57383
918
63932
919
35777
920
40209
921
40276
922
54945
923
59565
924
1776
925
64735
926
64737
927
64734
928
25325
929
56802
930
51736
931
17241
932
43936
933
5

In [6]:
s = pd.Series(beer_status)
s.to_csv("beer_status9.csv")


In [ ]:
# count number of beer reviews per beer
review_number = beer_df.groupby(["beer_name"]).count()

In [7]:
df = pd.DataFrame(beer_status)
df

,0
0,22579
1,Retired (no longer brewed)
2,22563
3,Retired (no longer brewed)
4,22564
...,...
1975,Retired (no longer brewed)
1976,69822
1977,Rotating/Seasonal
1978,31908


In [ ]:
# save to csv file for import
beer_data = df.to_csv("status.csv", header = True, compression="gzip", index = False)
